# **CNN 練習**


## 匯入所需套件

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

## Cifar10 資料讀入及前處理

In [ ]:
train_ds = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=T.ToTensor(),
)
test_ds = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=T.ToTensor(),
)
BATCH_SIZE = 512
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE,
                                           pin_memory=True,
                                           shuffle=True)
val_loader = torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE,
                                         pin_memory=True,
                                         shuffle=False)

## 模型定義
- 試著建立圖中的模型架構 (NOTE: Pytorch使用channel first順序)

e.g: (None, 32, 32, 3) -> (None, 3, 32, 32)

![image](https://hackmd.io/_uploads/By2IG1PL6.png)


In [ ]:
'''在__________填入正確的參數讓產生的卷積影像大小不變吧'''

model = nn.Sequential(
    # 建立卷積層，設定 32 個 3*3 的filters
    # 設定 padding，讓卷積運算，產生的卷積影像大小不變
    # 所有激活函數都設定為 ReLU
    nn.Conv2d(in_channels='______', out_channels='______', kernel_size='______', padding='______'),
    nn.'______',
    nn.Dropout(p=0.25),
    # 第二層 - 卷積層 (3x3 的 filters) + 池化層
    nn.Conv2d('______', '______', '______', padding='______'),
    nn.'______',
    nn.MaxPool2d(2),
    # 第三層 - 卷積層 (3x3 的 filters)
    nn.Conv2d('______', '______', '______', padding='______'),
    nn.'______',
    # 第四層 - 卷積層 (3x3 的 filters) + 池化層
    nn.Conv2d('______', '______', '______', padding='______'),
    nn.'______',
    nn.MaxPool2d(2),
    nn.Dropout(p=0.25),

    # 建立分類模型 (MLP) : 平坦層 + 隱藏層 (512 神經元, ReLU 為激活函數) + 輸出層 (10)
    nn.Flatten(),
    nn.Linear('______', '______'),
    nn.ReLU(),
    nn.Dropout(p=0.25),
    nn.Linear('______', 10)
)

model = model.to(device)

In [ ]:
from torchsummary import summary
summary(model, (3, 32, 32), device=device)

In [ ]:
def train_epoch(model, optimizer, loss_fn, train_dataloader, val_dataloader):
    # 訓練一輪
    model.train()
    total_train_loss = 0
    total_train_correct = 0
    for x, y in tqdm(train_dataloader, leave=False):
        optimizer.zero_grad() # 梯度歸零
        x, y = x.to(device), y.to(device) # 將資料移至GPU
        y_pred = model(x) # 計算預測值
        loss = loss_fn(y_pred, y) # 計算誤差
        loss.backward() # 反向傳播計算梯度
        optimizer.step() # 更新模型參數
        total_train_loss += loss.item()
        total_train_correct += ((y_pred.argmax(dim=1) == y).sum().item())

    avg_train_loss = total_train_loss / len(train_dataloader)
    avg_train_acc = total_train_correct / len(train_dataloader.dataset)

    return avg_train_loss, avg_train_acc

def test_epoch(model, loss_fn, val_dataloader):
    # 驗證一輪
    model.eval()
    total_val_loss = 0
    total_val_correct = 0
    with torch.no_grad():
        for x, y in val_dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            total_val_loss += loss.item()
            total_val_correct += ((y_pred.argmax(dim=1) == y).sum().item())

    avg_val_loss = total_val_loss / len(val_dataloader)
    avg_val_acc = total_val_correct / len(val_dataloader.dataset)

    return avg_val_loss, avg_val_acc

def run(epochs, model, optimizer, loss_fn, train_loader, valid_loader):
    train_loss_log = []
    val_loss_log = []
    train_acc_log = []
    val_acc_log = []
    best_val_loss = np.inf
    patience = 5
    stop_counter = 0
    for epoch in tqdm(range(epochs)):
        avg_train_loss, avg_train_acc = train_epoch(model, optimizer, loss_fn, train_loader, valid_loader)
        avg_val_loss, avg_val_acc = test_epoch(model, loss_fn, valid_loader)
        train_loss_log.append(avg_train_loss)
        val_loss_log.append(avg_val_loss)
        train_acc_log.append(avg_train_acc)
        val_acc_log.append(avg_val_acc)
        print(f'Epoch: {epoch}, Train Loss: {avg_train_loss:.3f}, Val Loss: {avg_val_loss:.3f} \
    | Train Acc: {avg_train_acc:.3f}, Val Acc: {avg_val_acc:.3f}')
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best.pth')
            stop_counter = 0
        else:
            stop_counter += 1
        if stop_counter == patience:
            print('Early stopping')
            break
    return train_loss_log, train_acc_log, val_loss_log, val_acc_log

In [ ]:
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

## 開始訓練模型

In [ ]:
logs = run(20, model, optimizer, loss_fn, train_loader, val_loader)

## 測試資料

In [ ]:
model.load_state_dict(torch.load('best.pth'))
model = model.eval()

In [ ]:
with torch.no_grad():
    img, label = test_ds[0]
    pred = model(img.unsqueeze(0).to(device))
    print('pred: ', pred.argmax(1))
    print('GT:   ', label)

plt.imshow(img.permute(1, 2, 0))
plt.show()

In [ ]:
loss, acc = test_epoch(model, loss_fn, val_loader)
print(f'Test Loss: {loss:.3f}, Test Acc: {acc:.3f}')

In [ ]:
# inference all test_loader
y_pred_list = []
y_true_list = []
with torch.no_grad():
    for x, y in val_loader:
        x = x.to(device)
        y_pred = model(x)
        y_pred_list.append(y_pred)
        y_true_list.append(y)

y_pred_list = torch.cat(y_pred_list).argmax(1).cpu()
y_true_list = torch.cat(y_true_list)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(f'Acc score: {accuracy_score(y_true_list, y_pred_list)}')
print(confusion_matrix(y_true_list, y_pred_list))

## 訓練結果視覺化

In [ ]:
train_history = ['loss', 'accuracy', 'val_loss', 'val_accuracy']
name_history = ['training_loss', 'val_loss', 'training_acc', 'val_acc']
train_loss, train_acc, val_loss, val_acc = logs
history = [train_loss, val_loss, train_acc, val_acc]

plt.figure(figsize=(12, 5))
for eachx, eachy, i in zip(history, name_history, range(4)):
    if i % 2 == 0:
        plt.subplot(1, 2, i//2+1)
    l_x = len(history[0])
    plt.plot(np.arange(l_x), history[i], label=eachy)
    plt.legend(loc='best')
    plt.title(eachy)
plt.show()